# Introduction to Learning Model Parmaters



## CSCI E-83
## Stephen Elston


In the previous lesson we examined several exact inference methods. When we applied these methods inference methods the parameters of the conditional probability distributions (CPD) were assumed known. But, how do we find these parameters in practice? 

In this lesson we will focus on the **learning** for Bayesian network models. There are two aspects of learning for Bayesian networks:  
1. **Parameter estimation** where the parameters of the CPDs of a model are estimated. We focus on this aspect of the problem in this notebook.   
2. **Structure estimation** is the process of estimating the structure of the graphical model. This problem is the subject of another notebook.  

The role of learning in an intelligent agent is illustrated in the figure below. For Bayesian models the learning and inference processes often merge as in the case where evidence is applied.  

<img src="img/Learning.JPG" alt="Drawing" style="width:400px; height:200px"/>
<center> **Learning in an intelligent agent** </center>

## Simple parameter estimation example

In previous lessons we used a belief network of a student looking for a job. As you will recall the student must submit her GRE score, $S$ and her letter of recommendation, $L$. Said that the probability of the GRE score (high, low) was conditional on the student's intelligence, $I$. We had a prior distribution of $I = [0.8, 0.2] = [p(high), p(low)]$. But, how do we estimate the parameters for this distribution?

There are several methods which can be used to estimate the parameters of a distribution including:
- Frequentist **maximum likelihood** or **ML** methods which only require that a likelihood function be specified. 
- Bayesian **maximum a-postiori** or **MAP** methods, which require the specification of both a likelihood function and a prior distribution of the model parameters. 

In this lesson we will focus on the MAP methods, but will also investigate ML methods. MAP methods are inherently Bayesian and therefore provide a more complete picture of distributions. However, MAP methods can be difficult to implement and computationally intensive. Thus in practice, ML methods are used. 



### The Bernoulli and Beta distributions

For this example, we will work with a binary variable with states {0,1}. A **single realization** of such a variable has a **Bernoulli distribution**. The Bernoulli distribution is a **parametric distribution** which we can express as:

$$p(\nu = 1) = \Theta \\
where\\
\nu = an\ observation\\
\Theta = probability\ parameter$$

For a series of observations of a binary valued variable we use the **Binomial distribution**. We call each observation where $\nu = 1$ a **success**. The Binomial distribution of $k$ successes in $n$ trials is then expressed as:

$$p(\nu = k\ |\ \Theta) = \binom{n}{k} \Theta^k (1-\Theta)^{n-k}$$





We want to specify a **Binomial likelihood** in Bayes theorem, we will also need to specify a prior distribution. We pick a prior distribution which is **conjugate** to the likelihood distribution so that the posterior distribution is the same as the prior. We can express this concept as:

$$posterior\_distribution(hypothesis\ |\ data) = \frac{likelihood(data\ |\ hypothesis)* conjugate\_prior(hypothesis)}{marginal\ distribtion\ data }\\
where\\
posterior\_distribution\ is\ same\ family\ as\ prior\_distribution$$

The question is, which distribution is the conjugate to the Binomial distribution? The answer is the **Beta distribution**. We can express the Beta distribution as: 

$$p(x\ |\ \alpha, \beta) = \frac{1}{B(\alpha, \beta)}x^{\alpha - 1}(1-x)^{\beta - 1},\ 0 \le x \le 1\\
where\\
B(\alpha, \beta) = \frac{\Gamma(\alpha) \Gamma(\beta)}{\Gamma(\alpha,\beta)}\\
and\\
\Gamma(x) = Gamma\ function$$

At first glance, the Beta distribution is a bit complicated. But for the most part the normalization with the Gamma function are not terribly important. 

You can develop some intuition about the Beta distribution by plotting it for various values of $\alpha$ and $\beta$. Execute the code in the cell below and examine the results. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import itertools as it
import scipy.stats as ss
%matplotlib inline

plt.figure(figsize=(12, 8))

alpha = [.5, 1, 2, 3, 4]
beta = alpha[:]
x = np.linspace(.001, .999, num=100)

for i, (a, b) in enumerate(it.product(alpha, beta)):
    plt.subplot(len(alpha), len(beta), i+1)
    plt.plot(x, ss.beta.pdf(x, a, b))
    plt.title('(a,b) = (%d,%d)' % (a,b))
plt.tight_layout()    

Some points to notice here:

- The range is $0 \le Beta(x) \le 1$.
- If $\alpha \gt \beta$ the distribution is left skewed, if $\alpha \lt \beta$ the distribution is right skewed, and if $\alpha = \beta$ the distribution is symmetric.
- For $\alpha = 1$, $\beta = 1$ the distribution is uniform. 

With these preliminary in mind, we will now look at methods to estimate the parameter of the Binomial likelihood distribution $\theta$.

### Kullback-Liebler Divergence

To train deep learning models **cross entropy** is often used as a loss function. This is an information theoretic measure of model fit. We can understand cross entropy as follows. 

First define **Shannon entropy** as:

$$\mathbb{H}(I) = E[I(X)] = E[-ln_b(P(X))] = - \sum_{i=1}^n P(x_i) ln_b(P(x_i))$$  
Where:  
$E[X] = $ the expectation of $X$.  
$I(X) = $ the information content of $X$.   
$P(X) = $ probability of $X$.  
$b = $ base of the logarithm.    

This rather abstract formula gives us a way to compute the expected information content of a set of values $X$. The more likely (higher probability) of $X$ the less informative it is. 

To create a loss function from the definition of Shannon entropy we start with the **Kullback-Leibler divergence (KL divergence)** or **relative entropy**. The KL divergence is an information theoretic measure of the difference between two distributions, $P(X)$ and $Q(X)$.

$$\mathbb{D}_{KL}(P \parallel Q) = - \sum_{i=1}^n p(x_i)\ ln_b \frac{p(x_i)}{q(x_i)}$$

Ideally, in the case of training a machine learning model we want a distribution $Q(X)$, which is identical to the actual data distribution $P(X)$. 

But, you may say, if we could know $P(X)$ why compute $Q(X)$ at all? Fortunately, we do not have to. We can rewrite the KL divergence as:

$$\mathbb{D}_{KL}(P \parallel Q) = \sum_{i=1}^n p(x_i)\ ln_b p(x_i) - \sum_{i=1}^n p(x_i)\ ln_b q(x_i)$$

Since $P(X)$ is fixed and we wish to find $Q(X)$ when we train our model, we can minimize the term on the right, which is the **cross entropy** defined as:

$$\mathbb{H}(P,Q) = - \sum_{i=1}^n p(x_i)\ ln_b q(x_i)$$

From the formulation of KL divergence above you can see the following.

$$\mathbb{D}_{KL}(P \parallel Q) = \mathbb{H}(P) + \mathbb{H}(P,Q)\\
\mathbb{D}_{KL}(P \parallel Q) = Entropy(P) + Cross\ Entropy(P,Q)$$

Thus, we can minimize divergence by minimizing cross entropy. This idea is both intuitive and computationally attractive. The closer the estimated distribution $q(X)$ is to the distribution of the true underling process $p(X)$, the lower the cross entropy and the lower the KL divergence. 

In general we will not know $p(X)$. In fact, if we did, why would we need to solve a training problem? So, we can use the following approximation.

$$\mathbb{H}(P,Q) = - \frac{1}{N} \sum_{i=1}^n ln_b q(x_i)$$

You may notice, that this approximation, using the average log likelihood, is equivalent to a maximum likelihood estimator (MLE). 

### Maximum Likelihood with K-L Divergence

We wish to find a distribution $q(x)$ with is identical to some unknown, and unknowable, data generating distribution $p^*(x)$. We can use the K-L divergence  

$$\mathbb{D}_{KL}(p^* \parallel q) = \sum_{i=1}^n p^*(x_i)\ ln_b \frac{p^* (x_i)}{q(x_i)} = 
- \mathbb{H}(p^*) - \mathbb{E}_{x \sim p^*} [log\ q(x)]$$

The first term only depends on $p^*$ so we do not need to consider this term. The maximum likelihood is achieved by maximizing the expected log-likelihood, $\mathbb{E}_{x \sim p^*} [log\ q(x)]$. This requires that the samples from $p^*$ reflect the true process distribution. Using this approach, we can compare distributions, $q(x)$, but never know if we have found the optimum.   

Finding the maximum of $\mathbb{E}_{x \sim p^*} [log\ q(x)]$ is still problematic, since we will never know $p^*$. But, we can use the empirical log-likelihood, for example as using a Monte Carlo estimate. The approach gives the following approximation: 

$$\mathbb{E}_{x \sim p^*} [log\ q(x)] = \frac{1}{|D|} \sum_{x \in D} log\ q(x)$$

Where, $D$ is the dataset drawn from the generating distribution $p^*$.

Given this approximation we can then find the maximum likelihood:

$$\underset{p \in D}{\max} \frac{1}{|D|} \sum_{x \in D} log\ q(x)$$

### Maximum likelihood method

We will not go into the details of **maximum likelihood estimation** (**MLE**). These details can be found in many standard texts on statistics and machine learning including Murphy (Section 7.3), Hastie, Tibshirani and Friedman (Section 2.6), and many others.  


Not surprisingly, in order to perform maximum likelihood estimation, we must define a **likelihood function** for the distribution we are trying to estimate. Given data $\mathcal{V}$ and parameters of the distribution, $\Theta$, we can express a likelihood function as $\mathcal{L}(\Theta, \mathcal{V}) = p(\mathcal{V}\ |\ \Theta)$. In most cases, we will work with the **log likelihood**, $log(p(\mathcal{V}\ |\ \Theta))$. The maximum likelihood estimate or MLE for the parameters is then:

$$\hat{\Theta} \doteq argmax_{\Theta} \{ \log(p(\mathcal{V}\ |\ \Theta)) \}$$

In words, we want to find the parameters which maximize the likelihood of the distribution for the data sample. 

Let's use computing the maximum likelihood for the Bernoulli distribution as an example. The likelihood function for the Bernoulli distribution is given a data vector $\mathcal{V}$:

$$L(\Theta) = \prod_{i=1}^n \Theta^{\nu_i}\ (1-\Theta)^{(1 - \nu_i)})$$

The log likelihood is easily computed:

$$\log({L}(\Theta)) = \log(\Theta) \sum_{i=1}^n \nu_i + \log(1-\Theta)  \sum_{i=1}^n (1 - \nu_i)$$

To find the maximum of this function we must find a point where the first partial derivative is zero:

$$\frac{\partial \mathcal{l}(\Theta)}{\partial \Theta} = \frac{\sum_{i=1}^n \nu_i}{\Theta} 
- \frac{\sum_{i=1}^n (1 - \nu_i)}{1-\Theta} = 0$$

The solution is: 

$$\Theta = \frac{1}{n} \sum_{i=1}^n \nu_i$$

The simple result is that the maximum likelihood value of $\Theta$ is just the mean of the data. In simple terms, we can write this relationship:

$$\Theta = \frac{count\ of\ successes}{total\ count}$$

You can verify that the second partial derivative is negative, indicating this is the maximum of the log likelihood function. 

### Bayesian learning

We can now try the Bayesian approach to estimating the parameter $\Theta$. Conceptually you can visualize this process as a directed Bayesian model as shown below. 

<img src="img/PlateDiagram.JPG" alt="Drawing" style="width:400px; height:200px"/>
<center> **Bayes network and plate diagram for parameter estimation** </center>

The parameter $\Theta$ is causal to the vector of data values $\vec{\nu} = \{ \nu^1, \nu^2, \nu^3, \ldots \nu^N \}$. This relationship is shown as a DAG on the left side of the figure above. On the right side of the diagram is a summary using **plate notation**. The plate is short hand for the $N$ values. 

The unnormalized conditional probability of the parameter $\Theta$ given the data vector $\vec{\nu}$ is: 

$$p(\Theta\ |\ \nu^1, \nu^2, \nu^3, \ldots \nu^N) \propto p(\Theta) \prod_{n=1}^N p(\nu^n\ |\ \Theta)
= \propto p(\Theta) \prod_{n=1}^N \Theta^{I(\nu^n=1)}(1 - \Theta)^{I(\nu^n=0)}\\
\propto p(\Theta)\ \Theta^{\sum_{n=1}^N I(\nu^n=1)}(1 - \Theta)^{\sum_{n=1}^N I(\nu^n=0)} $$

For a simple binary case of $\{ True, False \}$ the solution can be obtained with the prior distribution $p(\Theta)$ and the counts, $\{ N_T, N_F \}$.

$$p(\Theta\ |\ \nu^1, \nu^2, \nu^3, \ldots \nu^N) \propto p(\Theta)\ \Theta^{N_T} (1 - \Theta)^{N_F}$$  
Given a data vector, $\mathcal{V}$, how do we compute the most probable value of $\Theta$. The solution in this case is relatively easy and can be performed using just counts of the two values, {True, False} or {1,0} and the prior distribution. 

The likelihood given counts can be expressed as follows.

$$\Theta^{\mathcal{C}(\nu_i = 1)} (1-\Theta)^{\mathcal{C}(\nu_i = 0)}\\
where\\
\mathcal{C} = count\ operator$$

Using the above likelihood, the posterior distribution can be expressed as:

$$p(\Theta\ |\ \mathcal{V}) \propto p(\Theta)\ \Theta^{\mathcal{C}(\nu_i = 1)} (1-\Theta)^{\mathcal{C}(\nu_i = 0)}$$


But, how do we specify the prior distribution, $P(\Theta)$? The Beta distribution is the **conjugate** of the Binomial distribution. A the product of a distribution and its conjugate is in the family of the original distribution. Thus, it is both a theoretical and practical nicety to formulate a prior as the conjugate. 

Recall that the Beta distribution which has two parameters $\alpha$ and $\beta$:

$$p(x\ |\ \alpha, \beta) = \frac{1}{B(\alpha, \beta)}x^{\alpha - 1}(1-x)^{\beta - 1}$$

Given the Beta distribution we can specify the prior distribution in terms of **pseudo counts**. The pseudo count of successes is the value of the parameter $\alpha$. The pseudo count of failures is the value of the parameter $\beta$. 

Using the Beta prior, the posterior distribution of the can be written:

$$p(\Theta\ |\ \nu^1, \nu^2, \nu^3, \ldots \nu^N) 
\propto  \Theta^{\big( \sum_{n=1}^N I(\nu^n=1) \big)\ + \alpha }(1 - \Theta)^{\big( \sum_{n=1}^N I(\nu^n=0) \big)\ + \beta } $$

As with the maximum likelihood solution we can find the **maximum a posteriori (MAP)** value of $\Theta$ as:

$$\Theta = \frac{count\ of\ successes + \alpha}{total\ count + \alpha + \beta}$$

Some properties of the posterior given the Beta prior distributed prior include:
- The higher the ratio $\frac{\alpha}{\beta}$ the larger the prior value of $\Theta$.
- The larger the sum of $\alpha$ and $Beta$ the stronger the prior. Small values of $\alpha$ and $Beta$ specify a vague prior. 
- The larger the actual counts, the less important the prior is in determining $\Theta$.
- If $\alpha = \beta = 0$ the Bayesian estimate of $\Theta$ is identical to the maximum likelihood estimate. 
- If there are no actual counts, the estimate of $\Theta$ is the prior. 
- The prior regularizes the estimation of Q, even for variables with no observations.

Further details can be found in Murphy, Section 3.3 or Barber, Section 9.4. 

## Computational Example

Now, its time to put all this theory into practice with a computational example. We will continue to work with the student example illustrated below:

<img src="img/LetterDAG.JPG" alt="Drawing" style="width:400px; height:200px"/>
<center> **DAG CPDs for student example** </center>

In this example we will perform the following:

1. Simulate data for the CPDs of the model.
2. Test the CDPs by estimating the parameters using both maximum likelihood and maximum a posteriori methods.
3. Estimate the parameters for the entire model using both MLE and MAP methods. We will be able to examine the effects of different prior distributions. 

### Simulation of CPDs

Creating, testing and debugging simulation software can be tricky. This is particularly the case when conditional distributions are required. Fortunately, this process is greatly simplified through the use of a DAGs.   

Like many other probabilistic graphical model packages pgmpy has a simulation capability. Given a DAG model of class BayesianModel, the `BayesianModelSampling` class computes random samples. The code in the cell below reads a picked model file and computes simulated data from the distribution represented by the model.          

In [ ]:
## Simulate the binary tables
import numpy as np
import numpy.random as nr
import numpy as np
import pandas as pd
from pgmpy.sampling import BayesianModelSampling
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator
import pickle

## Load the model from a file
with open('student_model.pickle', 'rb') as pkl:
    murder_model = pickle.load(pkl)
print('The model loaded correctly: {}'.format(murder_model.check_model()))

## Simulate values from the DAG
def simulate_from_DAG(model, nsamps = 25, set_seed = 234):
    nr.seed(set_seed)
    simulation = BayesianModelSampling(model)
    return(simulation.forward_sample(size = nsamps, return_type='dataframe'))

samples_25 = simulate_from_DAG(murder_model)
samples_25

The results of above simulation looks reasonable. 

### MLE

But, what happens when we try to estimate the parameters of the distribution? The code in the cell below does the following:

1. Constructs a DAG with a single node.
2. Performs MLE for the model parameters and displays the results

In [ ]:
test_model = BayesianModel()
test_model.add_node('D')

# Fitting the data to the model using Maximum Likelihood Estimator
test_model.fit(pd.DataFrame(samples_25.loc[:,'D']), estimator=MaximumLikelihoodEstimator)
print(test_model.get_cpds('D'))

The MLE of the parameters are close to, but not exactly the same as, the value we used for the simulation, [0.1, 0.9]. 

### MAP

Next, we will try a Bayesian estimator. A Dirichlet distribution is used as the prior. We will examine the properties of this distribution in more detail shortly. 

The prior of a Dirichlet distribution is specified as a set of **pseudo counts**. As with any Bayesian estimation problem, we must face the issue of specification of the prior distribution. For this example, we could use any one, or perhaps a combination of the following approaches:
1. Select pseudo counts from previous data sets for the same or similar problems. 
2. Apply expert judgment. 
3. Use an uninformative prior, such as a uniform distribution, in which case the pseudo counts are equal. 

The code in the cell below uses the last approach, a uniform distribution. 

In [ ]:
# Fitting the data to the model using Maximum Likelihood Estimator
pseudo_counts = {'D': [[4], [6]]}
test_model.fit(pd.DataFrame(samples_25.loc[:,'D']), estimator=BayesianEstimator, prior_type='dirichlet', pseudo_counts=pseudo_counts)
print(test_model.get_cpds('D'))

The MAP estimate of parameters are close to know values and considerably different from the prior. This is expected since the sum of the pseudo counts sum to significantly less than the 25 data points. 



## Global and Local Parameter Independence

In the foregoing we have only considered the univariate Binomial and Beta distributions. But, real-world problems have many parameters. We need a way to deal with this situation without the mathematical and computational complexity of the full multivariate problem. 

Our approach is to factorize the joint distribution to introduce **global independence** and **local independence**. The global independence assumption allows us to factorize the **posterior distribution** over the conditional tables. The local independence assumption allows us to factor the prior distribution, a necessary condition for the global independence assumption. 

### Global independence assumption

Let's start with the global independence assumption using an example. Recalling the student example of previous lessons, let's consider the parameterized joint distribution of the student's grade in the machine learning course. Using plate notation we call illustrate this part of the DAG as shown here.

<img src="img/Factorizing.JPG" alt="Drawing" style="width:200px; height:300px"/>
<center> Plate diagram for factorizing joint distribution $p(\Theta_D, \Theta_I, \Theta_G)$ </center>

Let's assume we can factorize the joint distribution as follows.

$$p(\Theta_D, \Theta_I, \Theta_G) = p(\Theta_D)p(\Theta_I)p(\Theta_G)$$

Were $\Theta_D$, $\Theta_I$, and $\Theta_G$ are parameters of the distributions of difficulty, intelligence and grade respectively. 

If we assume the table (vector) of N data values, $\mathcal{X}$, are i.i.d. we can expand the above as:

$$p(\Theta_D, \Theta_I, \Theta_G) = p(\Theta_D)p(\Theta_I)p(\Theta_G) \prod_{n=1}^N p(d^n\ |\ \Theta_D)p(i^n\ |\ \Theta_I)p(g^n\ |\ d^n, i^n, \Theta_G)$$



### Local parameter independence

The local independence assumption allows us to factorize the prior distribution. Let's say that difficulty, $D$, and intelligence $I$, are only binomially distributed. We can factorize the prior distribution. For example: 

$$p(\Theta_G) = p(\Theta_G)^{0,0}p(\Theta_G)^{0,1}p(\Theta_G)^{1,0}p(\Theta_G)^{1,1}$$

Given this factorization we can find a factorization of the posterior distribution. 

$$p(\Theta_G\ |\ \mathcal{X}) \propto p(\mathcal{X}\ |\ \Theta_G) p(\Theta_G)^{0,0}p(\Theta_G)^{0,1}p(\Theta_G)^{1,0}p(\Theta_G)^{1,1}$$

## Introduction to the Categorical and Dirichlet Distributions

Many problems have multiple states or categories. The probabilities of the classes or categories are modeled with the **categorical distribution**. The categorical distribution is the multi-class generalization of the Bernoulli distribution. In fact, this distribution is sometime referred to as the **multinoulli distribution**. For Bayesian learning in these cases, the prior and posterior are modeled with the **Dirichlet distribution**. 

### The Categorical Distribution

We can express the **probability mass** function of the categorical distribution, with $k$ categories, with probability $p_i$ for the ith category as follows.

$$f(x = i\ |\ \boldsymbol{p}) = p_i \\
where\\
\boldsymbol{p} = (p_1, \ldots, p_k),\ and\\ \sum_{i=1}^k p_i = 1.0$$

An alternative formulation uses **Iverson bracket notation**. In this formulation $[x = i] = 1$ and $0$ otherwise. Using this notation the density function of the categorical distribution can be written:

$$f(x = i\ |\ \boldsymbol{p}) = \prod_{i=1}^k p_i^{[x = i]}$$



### The Dirichlet Distribution

In order to perform Bayesian learning with the categorical distribution we need to use Dirichlet distribution as a prior. However, it can be a bit difficult to develop intuition around the behavior of the Dirichlet distribution. We can express the Dirichlet distribution with $n$ categories as follows.

$$f(x_1, x_2, \ldots, x_n; \alpha_1, \alpha_2, \ldots, \alpha_n) = \frac{1}{B(\alpha)} \prod_{i=1}^{n} x_i^{\alpha_i - 1}$$
Where the normalization is the **Beta function**:
$$B(\alpha) = \frac{\prod_{i=1}^{n} \Gamma (\alpha_i) }{ \sum_{i=1}^{n} \Gamma ( \alpha_i )}$$
And $\Gamma( x )$ is the Gamma function. 

For our case of interest, the probabilities of the categories are represented by $\{ x_1, x_2, \ldots, x_n \}$ and are constrained by:    

\begin{align}
\sum_{i=1}^{n} x_i &= 1\\ 
x_i &\ge 0
\end{align}

In fact, in classification problems there is only one category per case, so $[x_i = 1]$ only for the ith category.  

The **concentration parameters** $\{ \alpha_1, \alpha_2, \ldots, \alpha_n \}$, determines how *concentrated* the the distribution is for each class. For the Dirichlet distribution, **prior information** is expressed in terms of the concentration hyperparamters $\boldsymbol{\alpha} = \{ \alpha_1, \alpha_2, \ldots, \alpha_n \}$. The values of $\alpha_i$ are parameterized in terms of **pseudocounts** for each category. The pseudocounts express prior information in two ways:
- Categories with larger pseudocounts have higher prior probabilities. 
- The total sum of pseudocounts expresses the strength of the prior. If the sum of all pseudocounts is relatively large with respect to the number of actual data, the prior will be strong.
- If all $\alpha_i$ are equal the Dirichlet distribution is uniform in the probabilities of the classes.



### Bayesian Parameter Estimation

When computing the Bayesian MAP of $\boldsymbol{p} = (p_1, \ldots, p_k)$ the posterior is Dirichlet distributed. We will skip the considerable amount of algebra and just state that for a category, $i$, with $c_i$ counts out of $N$ total data or count values values, $\boldsymbol{X} = (c_1, c_2, \ldots, c_n)$, the expected posterior MAP of $p_i$ can be expressed:

$$E(p_i\ |\ \boldsymbol{X}, \boldsymbol{\alpha}) = \frac{c_i + \alpha_i}{N + \sum_k \alpha_k}$$

The above expression has an intuitive explanation. Consider the following:
- The probability of a category $i$ increases as the count, $c_i$, increases relative to $N$, the sum of the counts for all categories. 
- As N increases relative to $\sum_k \alpha_k$ the data comes dominate the prior. On the other hand, if there are few actual data the prior will dominate. 



### Maximum Likelihood Estimation

Computing the MLE for the categorical distribution is rather strait forward. The expression is the same as for MAP, but with all $\alpha_i = 0$:

$$E(p_i\ |\ \boldsymbol{X}) = \frac{c_i}{N}$$

In other words, MLE probability for each category is just the faction of counts for that category in the dataset. 

## Computing Parameters for a Full Model

We can simultaneously estimate the parameters for an entire model. There are a total of 26 model parameters which must be fit using the 25 data points. Not only, but the distribution of the cases are not uniform given the sampling. This model is seriously **under-determined**. This situation will expose one of the significant weaknesses of MLE.    

The code in the cell below performs the following steps for the MLE of the model parameters:
1. The structure of the DAG was defined.
2. The parameters of the model are estimated using the `MaximumLikelihoodEstimator` in the `fit` method. 
3. The parameters of the CDPs are printed. 

In [ ]:
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator

student_model = BayesianModel([('D', 'G'), ('I', 'G'), ('G', 'L'), ('I', 'S')])

# Learing CPDs using Maximum Likelihood Estimators
student_model.fit(samples_25, estimator=MaximumLikelihoodEstimator)
for cpd in student_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)

The estimates of the conditionally independent variables are mostly as expected. Some of the other CDPs look a bit odd. Notice the 0s in the G, L and S variables. It is possible that much of this behavior is attributable to the under-determined nature of the problem. 

### Bayesian Estimation

Next, we will turn our attention to Bayesian parameter estimation. Recall that we performed Bayesian MAP estimation on the variable D using pseudo counts of $\{4, 6 \}$, a uniform and modestly strong prior.   

Let's explore the effect of using different priors. In the cell below, the parameters of the variable D are estimated using a strong prior, with pseudo counts $\{ 50, 50 \}$. This is considered a strong prior since the pseudo counts are larger than the total number of data cases. 

In [ ]:
test_model = BayesianModel()
test_model.add_node('D')

# Fitting the data to the model using Bayesian Estimator
pseudo_counts = {'D': [[50], [50]]}
test_model.fit(samples_25, estimator=BayesianEstimator, prior_type='dirichlet', pseudo_counts=pseudo_counts)
print(test_model.get_cpds('D'))

With the strong prior, the parameter estimates are close to the proportion of the pseudo counts. That is close to the prior. 


Next, we can try the parameters ................**************************

In [ ]:
student_model = BayesianModel([('D', 'G'), ('I', 'G'), ('G', 'L'), ('I', 'S')])

# Learing CPDs using Bayesian Estimators
pseudo_counts = {'D': [[4], [6]], 'I':[[4],[6]], 'G':[[5,2,8,3],[5,5,5,5],[5,8,2,7]], 'S':[[6,4],[4,6]], 'L':[[7,5,3],[3,5,7]]}
student_model.fit(samples_25, estimator=BayesianEstimator, prior_type='dirichlet', pseudo_counts=pseudo_counts)
for cpd in student_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)

Notice the following:
- The parameters of the conditionally independent variables, D and I, are nearly the same as obtained with the MLE method.  
- The parameter estimates for the variable G now seem more as we might expect. 
- The parameter estimates for the variables L and S appear to reflect the prior values of the parameters. 

The stabilization of the parameter estimates for the variable G is an example of **Bayesian regularization**. 

### MLE with more data

You may wonder if simply acquiring more data might help the MLE method behave better. In other words, is the small number of cases the only reason for poor MLE performance. Let's explore this aspect of the problem by simulating 10 times as much data. The code in the cell below simulates 250 cases using the same probabilities as before.

In [ ]:
## Let's try a larger sample!
nsamps = 250
samples_250 = simulate_from_DAG(murder_model, nsamps = nsamps)
samples_250

Now, we will compute the parameters for a model using the 250 data cases using MLE.

In [ ]:
student_model.fit(samples_250, estimator=MaximumLikelihoodEstimator)
for cpd in student_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)

In [ ]:
student_model.fit(samples_250, estimator=BayesianEstimator, prior_type='dirichlet', pseudo_counts=pseudo_counts)
for cpd in student_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)

These results are not much different from the ones obtained with 25 cases in the dataset. Evidently, lack of data was not the key problem. Rather, the structure of the model could be the limitation. 

#### Copyright 2018, 2019, Stephen F Elston. All rights reserved.

In [ ]:
def print_CPDS(model, decimals = 2):
    for CPT in student_model.cpds:
        print(CPT.variable)
        print(np.around(CPT.values, decimals=decimals))
        
print_CPDS(student_model)        

In [ ]:
student_model.cpds[1].__dict__